In [2]:
from dash import Dash, html, dcc

import dash_bootstrap_components as dbc

In [33]:
app = Dash(__name__)

app.layout = dbc.Container([
    # Divided the dashboard horizontally, and by default, 
    # the height of its child containers will be 100% of 
    # the parent. Can give width as pixels or percentages.
    html.Div([
        # Divided into 4 sections, one for title, another
        # for modes of operation buttons (spectrum, classification)
        # another for options (cluster_num, normalization, etc...)
        # and lastly a plot showing either classification or spectral
        # images.
        html.Div([
            html.H1("Spectral UI"),
            html.P([
                html.Br(),
                html.Span("PROJECT: [project-id]"),
                html.Br(),
                html.Span("SAMPLE: [sample-id]"),
                html.Br(),
                html.Br(),
                html.Span("INESC TEC Spectroscopy Group"),
            ])],
            style = {
                "vertical-alignment": "top",
                "height": 260
            }
        ), 
        html.Div(),
        html.Div(), 
        html.Div()
    ],
        style = {
        'width': "35%",
        'margin-left': 35,
        'margin-top': 35,
        'margin-bottom': 35
    }), 
    html.Div([
        # Divided into 2 sections, one for periodic table
        # and another for an average spectrum.
        html.Div(),
        html.Div()
    ],
        style = {
        'width': "65%",
        'margin-top': 35,
        'margin-right': 35,
        'margin-bottom': 35
    })
    ], 
    fluid = True,
    className = 'Dashboard-container'
    )

if __name__ == '__main__':
    app.run_server(debug = True)